# Perform detective work to find layer and features for a specific image and label

It may be useful to have the network diagram available as you trace backwards through the network

### Set up

In [1]:
# Set caffe root, label map, model definition, and model weights
#caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
caffe_root = '/work/caffe'
voc_labelmap_file = 'data/VOC0712/labelmap_voc.prototxt'
model_def = 'models/VGGNet/VOC0712/SSD_300x300/test.prototxt'
model_weights = 'models/VGGNet/VOC0712/SSD_300x300/VGG_VOC0712_SSD_300x300_iter_60000.caffemodel'

# Set confidence threshold (0-1) for object detection
conf_thresh = 0.6

# Set index of image of interest
image_index = 0

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Make sure that caffe is on the python path:
import os
os.chdir(caffe_root)
import sys
sys.path.insert(0, 'python')

import caffe
caffe.set_device(0)
caffe.set_mode_gpu()
from google.protobuf import text_format
from caffe.proto import caffe_pb2

# load PASCAL VOC labels
file = open(voc_labelmap_file, 'r')
voc_labelmap = caffe_pb2.LabelMap()
text_format.Merge(str(file.read()), voc_labelmap)

def get_labelname(labelmap, labels):
    num_labels = len(labelmap.item)
    labelnames = []
    if type(labels) is not list:
        labels = [labels]
    for label in labels:
        found = False
        for i in xrange(0, num_labels):
            if label == labelmap.item[i].label:
                found = True
                labelnames.append(labelmap.item[i].display_name)
                break
        assert found == True
    return labelnames

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))
transformer.set_mean('data', np.array([104,117,123])) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

### Iterate to image of interest

In [2]:
# iterate to image of interest
for i in range(0, image_index + 1):
    net.forward()

detections = net.blobs['detection_out'].data

# Parse the outputs.
det_label = detections[0,0,:,1]
det_conf = detections[0,0,:,2]
det_xmin = detections[0,0,:,3]
det_ymin = detections[0,0,:,4]
det_xmax = detections[0,0,:,5]
det_ymax = detections[0,0,:,6]

# Get detections with confidence higher than 0.6.
top_indices = [i for i, conf in enumerate(det_conf) if conf >= conf_thresh]

top_conf = det_conf[top_indices]
top_label_indices = det_label[top_indices].tolist()
top_labels = get_labelname(voc_labelmap, top_label_indices)
top_xmin = det_xmin[top_indices]
top_ymin = det_ymin[top_indices]
top_xmax = det_xmax[top_indices]
top_ymax = det_ymax[top_indices]

### Print number of high confidence objects in image

In [3]:
numel = top_conf.size;
print "num el:", str(numel)

num el: 1


### Find layer responsible for object

In [4]:
# Set k to the object of interest
k = numel - 1    # numel - 1 = most confident

top_conf_k = top_conf[k]
print "object", str(k),  "of", str(numel-1)
print "object confidence:", str(top_conf_k)
print "predicted label:",  top_labels[k]
print

# find index of this confidence in output of mbox_conf_softmax
conf_softmax = net.blobs['mbox_conf_softmax'].data
tconf_softmax_idx = np.where(conf_softmax == top_conf_k)
print "softmax layer shape:", conf_softmax.shape
print "index in softmax layer is " + str(tconf_softmax_idx)
print

# use index from  mbox_conf_softmax to find value in mbox_conf_reshape
conf_reshape = net.blobs['mbox_conf_reshape'].data
tconf_reshape = conf_reshape[tconf_softmax_idx[0], tconf_softmax_idx[1], tconf_softmax_idx[2]]

# use value from mbox_conf_reshape to find input layer (after perm) and index of layer
layers = ["conv4_3_norm_mbox_conf_perm", "fc7_mbox_conf_perm", "conv6_2_mbox_conf_perm", "conv7_2_mbox_conf_perm", "conv8_2_mbox_conf_perm",  "pool6_mbox_conf_perm"]

for layer in layers:
    layer_data = net.blobs[layer].data
    if tconf_reshape in layer_data:
        conf_perm = layer_data
        conf_perm_name = layer

conf_perm_idx = np.where(conf_perm == tconf_reshape)

print "output confidence came from layer " + conf_perm_name
print
print "permuted later shape:", conf_perm.shape
print "index in permuted is     " + str(conf_perm_idx)

# get index of data before permutation using prototxt info (0,1,2,3 -> 0,2,3,1)
# this is the index after convolving/pooling/fully connected layer
conf_idx = [conf_perm_idx[0], conf_perm_idx[3], conf_perm_idx[1], conf_perm_idx[2]]
print "index before premuted is " + str(conf_idx)
print
# get data from layer after conv/pool/fc
conf_post_name = conf_perm_name[:-5]
conf_post = net.blobs[conf_post_name].data

# get bin high conf is found in
bin = conf_idx[1] / 21

# IMGAGE FEATURE LAYER AND INDECES
print "--Layer With Image Features--"
print "name of conv/pool/fc conf layer is " + conf_post_name
tconf_post = conf_post[conf_idx[0], conf_idx[1], conf_idx[2], conf_idx[3]]
print "size of conf layer is " + str(net.blobs[conf_post_name].data.shape)
print "image features found at indeces (" + str(int(conf_idx[0])) + "," + str(int(conf_idx[1])) + "," + str(int(conf_idx[2])) + "," + str(int(conf_idx[3])) + ")"

object 0 of 0
object confidence: 0.999378
predicted label: person

softmax layer shape: (1, 7308, 21)
index in softmax layer is (array([0]), array([7273]), array([15]))

output confidence came from layer conv8_2_mbox_conf_perm

permuted later shape: (1, 3, 3, 126)
index in permuted is     (array([0]), array([1]), array([1]), array([36]))
index before premuted is [array([0]), array([36]), array([1]), array([1])]

--Layer With Image Features--
name of conv/pool/fc conf layer is conv8_2_mbox_conf
size of conf layer is (1, 126, 3, 3)
image features found at indeces (0,36,1,1)
